<a href="https://colab.research.google.com/github/Charles-Scott-Green/Argument-mining/blob/master/3_Baseline_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!python -m spacy download en_core_web_lg --quiet

     |████████████████████████████████| 826.9MB 63.3MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 2.7MB/s 


In [0]:
!pip install spacy-wordnet

     |████████████████████████████████| 655kB 2.7MB/s 
     |████████████████████████████████| 1.4MB 14.4MB/s 
  Created wheel for spacy-wordnet: filename=spacy_wordnet-0.0.4-py2.py3-none-any.whl size=650292 sha256=13aadb7c684bb7162ef055850a9c30a1baa12962ff2fff62b50539e4dd0f192d
  Stored in directory: /root/.cache/pip/wheels/25/93/1d/c86db913cd146fc9ddb26d10f56579c5d58a3e00bc8f96a3a6
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394470 sha256=67f01727ce9fef3fa55a377ebcd489f985481953fd5257e8c7618fdcad51ab93
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built spacy-wordnet nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


## Import Libraries and initialize settings

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [0]:
import re
import string
import math
from tqdm import tqdm

In [0]:
import nltk
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize, word_tokenize
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

In [0]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

In [0]:
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
from spacy.lang.en.stop_words import STOP_WORDS
from spacy_wordnet.wordnet_annotator import WordnetAnnotator

nlp = spacy.load("en_core_web_lg")
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')

In [0]:
import warnings
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
warnings.filterwarnings('ignore')

In [0]:
from sklearn.model_selection import train_test_split as tts 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb

from imblearn.over_sampling import SMOTE

In [0]:
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pickle

## Define Functions

In [0]:
def get_target_data(path):
    df = pd.read_csv(path)
    df['cdc_present'] = (df['cdc'] != '---').astype(int)

    return df['cdc_present']

In [0]:
def topic_domains(topic):
    raw_domains = []
    common_domains = []

    for chunk in nlp(topic).noun_chunks:
        rt_nn = chunk.root
        raw_domains.append(rt_nn._.wordnet.wordnet_domains())

    if len(raw_domains) > 0:
        reference = raw_domains[0]
    else:
        return list(raw_domains)
        
    for domain in raw_domains:
        common_domains = set(reference) & set(domain)
        
    return list(common_domains)

In [0]:
def expanded_sentence_similarity(topic, sentence):
    
    extended_topic = []
    domains = topic_domains(topic)
    
    if len(domains) < 1:
        return nlp(topic).similarity(nlp(sentence))
    else:
        for token in nlp(topic):
            synsets = token._.wordnet.wordnet_synsets_for_domain(domains)
            if synsets: 
                lfs = []
                for syn in synsets:
                    lfs.extend(syn.lemma_names())
                    extended_topic.append('({})'.format('|'.join(set(lfs))))
            else:
                extended_topic.append(token.text)

        tokenized_extended_topic = nlp(' '.join(extended_topic))

    return tokenized_extended_topic.similarity(nlp(sentence))            

In [0]:
def subject_match(topic, sentence):
    
    ntopic = nlp(topic)
    scores = []
    for chunk in nlp(sentence).noun_chunks:
        scores.append(chunk.similarity(ntopic))
    
    return np.mean(scores)

In [0]:
def subject_match2(topic, sentence):
    
    ntopic = nlp(topic)
    scores = []
    for chnk in nlp(sentence).noun_chunks:
        for tchnk in ntopic.noun_chunks:
            scores.append(chnk.similarity(tchnk))
    
    return np.mean(scores)

In [0]:
def includes_that(sentence):
    check = []
    for tok in sentence:
        if tok.tag_ == 'IN':
            check.append(tok.text)
    if 'that' in check:
        return 1
    else:
        return 0

In [0]:
def patterns(sentence, pattern_type='dep'):
    
    pat = []
    if pattern_type == 'dep':
        pat = [tok.dep_ for tok in sentence]
    elif pattern_type == 'tag':
        pat = [tok.tag_ for tok in sentence]
    
    return '|'.join(pat)

In [0]:
def pos_pattern(sentence):
    pat = []
    for tok in sentence:
        pat.append(tok.pos_)

    return '|'.join(pat)


In [0]:
def pattern_in_text(pattern, sentence):
    text_pattern = pos_pattern(sentence)

    if pattern in text_pattern:
        return 1
    else:
        return 0

In [0]:
def infinitive_verb_present(sentence):
    text_pattern = pos_pattern(sentence)
    infinitive_pattern = 'PART|VERB'

    if infinitive_pattern in text_pattern:
        return 1
    else:
        return 0


In [0]:
def sentiment_subjectivity(sentence, score_type='sentiment'):
    analyzer = SentimentIntensityAnalyzer()

    if score_type == 'sentiment':
        pos = analyzer.polarity_scores(sentence)['pos']
        neg = analyzer.polarity_scores(sentence)['neg']
        return pos/neg
    
    else:
        return analyzer.polarity_scores(sentence)['compound']

In [0]:
def is_sentence(clause):
    nn = 2
    vb = 1
    cls = nlp(clause)
    for tok in cls:
        if tok.pos_ in ['NOUN', 'PROPN', 'PRON']:
            nn -= 1
        elif tok.pos_ in ['VERB']:
            vb -= 1
    if nn < 1 and vb < 1:
        return True
    else:
        return False

In [0]:
def get_clauses(sentence):
    """Sentence type is a string."""
    subtree_phrases = []
    
    for tok in nlp(sentence):
        if tok.head.pos_ == 'VERB':
            if len([tok for tok in tok.subtree]) >= 3:
                subtree_prhase = ''.join(tok.text_with_ws for tok in tok.subtree)
                if subtree_prhase != sentence:
                    if is_sentence(subtree_prhase):
                        splits = subtree_prhase.split(',')
                        if len(splits) == 1:
                            subtree_phrases.append(subtree_prhase)
                        else:
                            for split in splits:
                                if is_sentence(split):
                                    if split not in subtree_phrases:
                                        subtree_phrases.append(split) 
    
    raw_clauses = []
    clauses = []

    for phrase in subtree_phrases:
        tok_phrase = nlp(phrase)
        for chunk in tok_phrase.noun_chunks:
            span = tok_phrase[chunk.root.head.left_edge.i:chunk.root.head.right_edge.i+1]
            if span[0].pos_ in ['DET', 'ADP']:
                if span[1:].text not in raw_clauses:
                    if is_sentence(span[1:].text):
                        raw_clauses.append(span[1:].text)
            elif span.text not in raw_clauses:
                if is_sentence(span.text):
                    raw_clauses.append(span.text)
    
    if len(raw_clauses) == 0:
        clauses.append(sentence)
    else:
        for clause in raw_clauses:
            dup_checks = [(clause in check_clause) for check_clause in raw_clauses]
            if sum(dup_checks) == 1:
                clauses.append(clause)
                
    return clauses

In [0]:
def conf_matrix(y_true, y_predict):
    
    from sklearn.metrics import confusion_matrix
    
    data = confusion_matrix(y_true, y_predict)
    index = ['Actual_0', 'Actual_1']
    columns = ['Predicted_0', 'Predicted_1']
    
    return pd.DataFrame(data, index, columns)

In [0]:
def get_target_data(path):
    df = pd.read_csv(path)
    df['cdc_present'] = (df['cdc'] != '---').astype(int)

    return df['cdc_present']

In [0]:
def count_sentiment_words(sentence):
    analyzer = SentimentIntensityAnalyzer()
    wd_cnt = 0

    for tok in nlp(sentence):
        if analyzer.polarity_scores(tok.text)['compound'] != 0:
            wd_cnt += 1

    return wd_cnt

In [0]:
def count_syllables(sentence):
    """Given a string, calculates and returns the number of syllables of the word."""
    import string
    
    word_list = sentence.translate(string.punctuation).split()
    vowels = "aeiouy"
    tally = 0
    
    for word in word_list:
        word = word.lower()
        vowels = "aeiouy"
        count = 0
        
        if word[0] in vowels:
            count += 1

        for i in range(1, len(word)):
            if word[i] in vowels and word[i - 1] not in vowels:
                count += 1

        if word.endswith('e'):
            count -= 1

        if count == 0:
            count += 1
    
        tally += count
    
    return tally

In [0]:
def punct_marks(sentence):
    """Given a sentence, returns count and/or list of punctiation marks"""
    
    pm = []
    
    for tok in nlp(sentence):
        if tok.pos_ == 'PUNCT':
            pm.append(tok)
    return len(pm)

In [0]:
def noun_counts(sentence):
    
    chnk = []
    
    for chunk in nlp(sentence).noun_chunks:
        chnk.append(chunk)

    return len(chnk)

In [0]:
def word_count(sentence):
    wds = []
    for tok in nlp(sentence):
        if tok.pos_ != 'PUNCT':
            wds.append(tok)
    return len(wds)

In [0]:
def phrase_chunks(sentence):
    s = []
    for tok in nlp(sentence):
        if len([tok for tok in tok.subtree]) >=3:
            if ''.join(tok.text_with_ws for tok in tok.subtree) != str(sentence):
                s.append(''.join(tok.text_with_ws for tok in tok.subtree))
    return len(s)

## Load Dataframe

In [0]:
# paths to data
path = '/content/drive/My Drive/Colab Notebooks/Thinkful/Module 34 - Final Capstone/data/CDC Detection/cleaned.csv'

df = pd.read_csv(path)

In [0]:
df.head()

,Unnamed: 0,Unnamed: 0.1,text,topic,article,cdc,cdc_present,context,text_words,text_syllables,text_sentiment_words,text_punctuation,text_nouns,text_phrases,sent_rat,clauses,num_clauses,claused_sentence,context_similarity,expanded_similarity,subjectivity_score
0,0,0,Deficit spending is the amount by which a gove...,Europe should weaken its austerity measures to...,Deficit spending,---,False,austerity measures,35,67,3,10,10,12,0.085714,"["" or individual's spending exceeds income ove...",1,or individual's spending exceeds income over ...,0.326836,0.259504,0.0000
1,1,1,Government deficit spending is a central point...,Europe should weaken its austerity measures to...,Deficit spending,---,False,austerity measures,14,29,1,2,4,5,0.071429,['Government deficit spending is a central poi...,1,Government deficit spending is a central point...,0.373849,0.267986,-0.4019
2,2,2,Government deficit spending is a central point...,Europe should weaken its austerity measures to...,Deficit spending,---,False,austerity measures,17,37,2,2,6,6,0.117647,['prominent economists holding differing views'],1,prominent economists holding differing views,0.267243,0.106333,0.3182
3,3,3,The mainstream economics position is that defi...,Europe should weaken its austerity measures to...,Deficit spending,---,False,austerity measures,66,122,7,7,15,22,0.106061,['deficit spending is desirable and necessary ...,4,deficit spending is desirable and necessary as...,0.461772,0.240641,-0.9280
4,4,4,"This is derived from Keynesian economics, and ...",Europe should weaken its austerity measures to...,Deficit spending,---,False,austerity measures,34,64,3,5,6,11,0.088235,['has been the mainstream economics view (in t...,1,has been the mainstream economics view (in the...,0.318571,0.306673,0.3612


In [0]:
# vectorize text
txt = [nlp(sentence) for sentence in df['text']]
v_txt = [t.vector for t in txt]

In [0]:
cxt = [nlp(sentence) for sentence in df['context']]
v_cxt = [t.vector for t in cxt]

## Preprocessing

In [0]:
txt_vects = pd.DataFrame(v_txt)
txt_vects.columns = ['txt_'+str(col) for col in txt_vects.columns]

cxt_vects = pd.DataFrame(v_cxt)
cxt_vects.columns = ['cxt_'+str(col) for col in cxt_vects.columns]

X = pd.concat([txt_vects, cxt_vects], axis=1)
y = df['cdc_present']

In [0]:
# Initialize SMOTE for class imbalance handling
sm = SMOTE(random_state=42)

# Split features,target into train/test sets
X_trn, X_tst, y_trn, y_tst = tts(txt_vects, y, test_size=0.25, random_state=42)

# resample train set
X_res, y_res = sm.fit_resample(X_trn, y_trn)


## Modeling

### Baseline

This initial model will use only vectorize full text and context features.

#### First pass

In [0]:
lr = LogisticRegression()
rf = RandomForestClassifier()
lb = lgb.LGBMClassifier()

lr.fit(X_res, y_res)
rf.fit(X_res, y_res)
lb.fit(X_res, y_res)

pn_lr = lr.predict(X_trn)
pt_lr = lr.predict(X_tst)
pn_rf = rf.predict(X_trn)
pt_rf = rf.predict(X_tst)
pn_lb = lb.predict(X_trn)
pt_lb = lb.predict(X_tst)

print("\n----------------------LR Scores----------------------")
print('Training set score:', lr.score(X_trn, y_trn))
print('Training precision:', precision_score(y_trn, pn_lr))
print('Training recall:', recall_score(y_trn, pn_lr))
print('\nTest set score:', lr.score(X_tst, y_tst))
print('Test precision:', precision_score(y_tst, pt_lr))
print('Test recall:', recall_score(y_tst, pt_lr))
print("\n----------------------LR Scores----------------------")
print('Training set score:', rf.score(X_trn, y_trn))
print('Training precision:', precision_score(y_trn, pn_rf))
print('Training recall:', recall_score(y_trn, pn_rf))
print('\nTest set score:', rf.score(X_tst, y_tst))
print('Test precision:', precision_score(y_tst, pt_rf))
print('Test recall:', recall_score(y_tst, pt_rf))
print("\n----------------------LR Scores----------------------")
print('Training set score:', lb.score(X_trn, y_trn))
print('Training precision:', precision_score(y_trn, pn_lb))
print('Training recall:', recall_score(y_trn, pn_lb))
print('\nTest set score:', lb.score(X_tst, y_tst))
print('Test precision:', precision_score(y_tst, pt_lb))
print('Test recall:', recall_score(y_tst, pt_lb))


----------------------LR Scores----------------------
Training set score: 0.8024064256008128
Training precision: 0.11493573644556064
Training recall: 0.7695351137487636

Test set score: 0.7951428571428572
Test precision: 0.1015625
Test recall: 0.7428571428571429

----------------------LR Scores----------------------
Training set score: 1.0
Training precision: 1.0
Training recall: 1.0

Test set score: 0.9614285714285714
Test precision: 0.3170731707317073
Test recall: 0.24761904761904763

----------------------LR Scores----------------------
Training set score: 0.9579669195847488
Training precision: 0.4281780633318036
Training recall: 0.9228486646884273

Test set score: 0.926
Test precision: 0.19117647058823528
Test recall: 0.45396825396825397


In [0]:
# Training set
conf_matrix(y_trn, pn_lr)

,Predicted_0,Predicted_1
Actual_0,24497,5991
Actual_1,233,778


In [0]:
# Test set
conf_matrix(y_tst, pt_lr)

,Predicted_0,Predicted_1
Actual_0,8115,2070
Actual_1,81,234


In [0]:
# Training set
conf_matrix(y_trn, pn_rf)

,Predicted_0,Predicted_1
Actual_0,30488,0
Actual_1,0,1011


In [0]:
# Test set
conf_matrix(y_tst, pt_rf)

,Predicted_0,Predicted_1
Actual_0,10017,168
Actual_1,237,78


In [0]:
# Training set
conf_matrix(y_trn, pn_lb)

,Predicted_0,Predicted_1
Actual_0,29242,1246
Actual_1,78,933


In [0]:
# Test set
conf_matrix(y_tst, pt_lb)

,Predicted_0,Predicted_1
Actual_0,9580,605
Actual_1,172,143


### Adding Features

At this point, recall of positive instances is more important than overall model accuracy.

Logistic Regression, with a recall score of .743, easily outperformed other models.  Additionally, it generalizes well when compared to its training scores.  Given how distinct the advantage Logistic Regression has over other models, in both overfitting and recall, it will likely be the selected model for this task.  

The next model will evaluate the use of vectorized full text, vectorized context, and the addition of other engineered features.

In [0]:
sc = StandardScaler()
F = pd.DataFrame()

F['txt_wds'] = df['text_words']
F['txt_syl'] = df['text_syllables']
F['snt_rat'] = df['text_sentiment_words'] / df['text_words']
F['txt_pct'] = df['text_punctuation']
F['txt_nns'] = df['text_nouns']
F['txt_phs'] = df['text_phrases']
F['tpc_sim'] = df['context_similarity']
F['exp_sim'] = df['expanded_similarity']
F['subj_sc'] = df['subjectivity_score']

In [0]:
sc_F = sc.fit_transform(F)

sc_F = pd.DataFrame(sc_F)

In [0]:
X = pd.concat([X, sc_F], axis=1)

In [0]:
X_trn, X_tst = X.iloc[X_trn.index], X.iloc[X_tst.index]

X_res, y_res = sm.fit_resample(X_trn, y_trn)

In [0]:
lr = LogisticRegression()
rf = RandomForestClassifier()
lb = lgb.LGBMClassifier()

lr.fit(X_res, y_res)
rf.fit(X_res, y_res)
lb.fit(X_res, y_res)

pn_lr = lr.predict(X_trn)
pt_lr = lr.predict(X_tst)
pn_rf = rf.predict(X_trn)
pt_rf = rf.predict(X_tst)
pn_lb = lb.predict(X_trn)
pt_lb = lb.predict(X_tst)

print("\n----------------------LR Scores----------------------")
print('Training set score:', lr.score(X_trn, y_trn))
print('Training precision:', precision_score(y_trn, pn_lr))
print('Training recall:', recall_score(y_trn, pn_lr))
print('\nTest set score:', lr.score(X_tst, y_tst))
print('Test precision:', precision_score(y_tst, pt_lr))
print('Test recall:', recall_score(y_tst, pt_lr))
print("\n----------------------RF Scores----------------------")
print('Training set score:', rf.score(X_trn, y_trn))
print('Training precision:', precision_score(y_trn, pn_rf))
print('Training recall:', recall_score(y_trn, pn_rf))
print('\nTest set score:', rf.score(X_tst, y_tst))
print('Test precision:', precision_score(y_tst, pt_rf))
print('Test recall:', recall_score(y_tst, pt_rf))
print("\n----------------------LB Scores----------------------")
print('Training set score:', lb.score(X_trn, y_trn))
print('Training precision:', precision_score(y_trn, pn_lb))
print('Training recall:', recall_score(y_trn, pn_lb))
print('\nTest set score:', lb.score(X_tst, y_tst))
print('Test precision:', precision_score(y_tst, pt_lb))
print('Test recall:', recall_score(y_tst, pt_lb))


----------------------LR Scores----------------------
Training set score: 0.8056128765992571
Training precision: 0.1221170904790065
Training recall: 0.8170128585558852

Test set score: 0.8044761904761905
Test precision: 0.1099640933572711
Test recall: 0.7777777777777778

----------------------RF Scores----------------------
Training set score: 1.0
Training precision: 1.0
Training recall: 1.0

Test set score: 0.9664761904761905
Test precision: 0.39664804469273746
Test recall: 0.2253968253968254

----------------------LB Scores----------------------
Training set score: 0.9827931045430014
Training precision: 0.7994891443167306
Training recall: 0.619188921859545

Test set score: 0.9661904761904762
Test precision: 0.38372093023255816
Test recall: 0.20952380952380953


In [0]:
# Training set
conf_matrix(y_trn, pn_lr)

,Predicted_0,Predicted_1
Actual_0,24550,5938
Actual_1,185,826


In [0]:
# Test set
conf_matrix(y_tst, pt_lr)

,Predicted_0,Predicted_1
Actual_0,8202,1983
Actual_1,70,245


In [0]:
# Training set
conf_matrix(y_trn, pn_rf)

,Predicted_0,Predicted_1
Actual_0,30488,0
Actual_1,0,1011


In [0]:
# Test set
conf_matrix(y_tst, pt_rf)

,Predicted_0,Predicted_1
Actual_0,10077,108
Actual_1,244,71


In [0]:
# Training set
conf_matrix(y_trn, pn_lb)

,Predicted_0,Predicted_1
Actual_0,30331,157
Actual_1,385,626


In [0]:
# Test set
conf_matrix(y_tst, pt_lb)

,Predicted_0,Predicted_1
Actual_0,10079,106
Actual_1,249,66


### Tuning the LR model

In [0]:
params = [{'C':[.001, .01, .1, 1, 10, 100, 1000, 1000],
           'penalty': ['l1', 'l2']}]

lr = LogisticRegression()

clf = GridSearchCV(lr, param_grid=params, scoring='recall', cv=3)
clf.fit(X_res, y_res)


GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 1000],
                          'penalty': ['l1', 'l2']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='recall', verbose=0)

In [0]:
clf.best_estimator_

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
lr = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
lr.fit(X_res, y_res)

pn_lr = lr.predict(X_trn)
pt_lr = lr.predict(X_tst)

print("\n----------------------LR Scores----------------------")
print('Training set score:', lr.score(X_trn, y_trn))
print('Training precision:', precision_score(y_trn, pn_lr))
print('Training recall:', recall_score(y_trn, pn_lr))
print('\nTest set score:', lr.score(X_tst, y_tst))
print('Test precision:', precision_score(y_tst, pt_lr))
print('Test recall:', recall_score(y_tst, pt_lr))


----------------------LR Scores----------------------
Training set score: 0.8056128765992571
Training precision: 0.1221170904790065
Training recall: 0.8170128585558852

Test set score: 0.8044761904761905
Test precision: 0.1099640933572711
Test recall: 0.7777777777777778


In [0]:
# Training set
conf_matrix(y_trn, pn_lr)

,Predicted_0,Predicted_1
Actual_0,24550,5938
Actual_1,185,826


In [0]:
# Test set
conf_matrix(y_tst, pt_lr)

,Predicted_0,Predicted_1
Actual_0,8202,1983
Actual_1,70,245


In [0]:
scores = cross_val_score(lr, X_res, y_res, cv=5)

In [0]:
print(scores)

[0.82732043 0.8400984  0.82542025 0.83714637 0.83583436]


This model will work serve as the first funnel.

In [0]:
X.head()

,txt_0,txt_1,txt_2,txt_3,txt_4,txt_5,txt_6,txt_7,txt_8,txt_9,txt_10,txt_11,txt_12,txt_13,txt_14,txt_15,txt_16,txt_17,txt_18,txt_19,txt_20,txt_21,txt_22,txt_23,txt_24,txt_25,txt_26,txt_27,txt_28,txt_29,txt_30,txt_31,txt_32,txt_33,txt_34,txt_35,txt_36,txt_37,txt_38,txt_39,...,cxt_269,cxt_270,cxt_271,cxt_272,cxt_273,cxt_274,cxt_275,cxt_276,cxt_277,cxt_278,cxt_279,cxt_280,cxt_281,cxt_282,cxt_283,cxt_284,cxt_285,cxt_286,cxt_287,cxt_288,cxt_289,cxt_290,cxt_291,cxt_292,cxt_293,cxt_294,cxt_295,cxt_296,cxt_297,cxt_298,cxt_299,0,1,2,3,4,5,6,7,8
0,-0.043492,0.255405,-0.031507,-0.062282,0.063146,-0.003994,-0.055570,-0.098684,-0.005464,2.366149,-0.296168,0.114405,0.087672,0.035144,-0.041646,-0.125251,0.001274,1.161077,-0.164186,0.085831,-0.037350,-0.019712,-0.124259,-0.046066,-0.035945,-0.025597,-0.090412,0.030220,-0.018963,-0.066032,-0.014213,0.064528,-0.089806,0.122765,0.135511,-0.083966,-0.071495,-0.004060,-0.063389,-0.081253,...,-0.030397,0.57278,0.05254,-0.308745,-0.091502,0.39261,0.34084,0.683425,0.379645,0.235475,0.219738,0.43815,-0.10628,0.207547,0.38671,0.024826,-0.157651,-0.046985,0.134404,-0.242321,0.095336,-0.091245,0.57893,0.619055,0.52503,0.454925,-0.026794,0.04523,-0.15126,0.45913,-0.178115,0.787621,0.976035,0.207532,2.286923,0.900853,0.524713,-0.108448,-0.258628,-0.019056
1,-0.109559,0.207248,0.102608,-0.002110,0.011729,-0.042757,-0.067221,-0.044486,-0.010553,2.539694,-0.263917,0.052020,0.154395,-0.017070,-0.051339,-0.101977,0.010399,1.085244,-0.170203,0.007699,0.008053,-0.038508,-0.041094,-0.032602,0.050750,-0.029007,-0.062326,0.109653,-0.122150,0.048205,-0.060107,0.189046,0.037814,-0.037662,0.189466,-0.040528,0.032219,0.022903,-0.104137,-0.135916,...,-0.030397,0.57278,0.05254,-0.308745,-0.091502,0.39261,0.34084,0.683425,0.379645,0.235475,0.219738,0.43815,-0.10628,0.207547,0.38671,0.024826,-0.157651,-0.046985,0.134404,-0.242321,0.095336,-0.091245,0.57893,0.619055,0.52503,0.454925,-0.026794,0.04523,-0.15126,0.45913,-0.178115,-0.824576,-0.637104,-0.014273,-0.489937,-0.712199,-0.689900,0.203428,-0.188807,-1.041112
2,-0.112302,0.123262,0.083034,-0.025925,0.002719,-0.095401,-0.076768,0.036539,0.031233,2.420226,-0.231760,-0.010759,0.091659,-0.001610,-0.061019,-0.143259,-0.024492,0.957714,-0.172521,0.039617,-0.028506,-0.033144,0.025336,-0.048492,0.125618,-0.069546,-0.002470,0.151607,-0.083742,0.070271,-0.038452,0.292199,0.002647,-0.160415,0.203062,-0.034336,0.050418,-0.041113,-0.063184,-0.165722,...,-0.030397,0.57278,0.05254,-0.308745,-0.091502,0.39261,0.34084,0.683425,0.379645,0.235475,0.219738,0.43815,-0.10628,0.207547,0.38671,0.024826,-0.157651,-0.046985,0.134404,-0.242321,0.095336,-0.091245,0.57893,0.619055,0.52503,0.454925,-0.026794,0.04523,-0.15126,0.45913,-0.178115,-0.594262,-0.297496,0.703333,-0.489937,-0.174515,-0.516384,-0.503771,-1.519513,0.790146
3,-0.053385,0.214746,0.021882,0.015795,-0.019167,-0.045329,-0.057965,0.084099,0.032091,2.271004,-0.239476,0.045541,0.127291,0.108474,-0.017606,-0.054341,0.038934,1.124100,-0.146744,0.135011,-0.047707,0.024707,-0.048830,-0.061580,-0.035538,0.026508,-0.057374,0.039397,-0.038100,-0.037311,-0.100791,0.107844,-0.016423,-0.053083,0.109905,-0.058154,0.046641,-0.029522,-0.051385,-0.118465,...,-0.030397,0.57278,0.05254,-0.308745,-0.091502,0.39261,0.34084,0.683425,0.379645,0.235475,0.219738,0.43815,-0.10628,0.207547,0.38671,0.024826,-0.157651,-0.046985,0.134404,-0.242321,0.095336,-0.091245,0.57893,0.619055,0.52503,0.454925,-0.026794,0.04523,-0.15126,0.45913,-0.178115,3.167530,3.310840,0.523437,1.245600,2.245064,2.259873,0.786688,-0.413902,-2.379016
4,-0.018335,0.202566,0.023854,0.021207,0.018555,0.012194,-0.049137,-0.005782,-0.034290,2.005933,-0.227361,0.048698,0.150188,-0.014524,0.057379,-0.038083,-0.131232,0.898381,-0.062469,-0.040641,-0.048399,-0.122306,0.013465,0.038691,0.056038,-0.046874,0.007244,0.108148,-0.133234,0.110886,-0.031193,0.194429,0.019248,-0.083311,0.123750,-0.046843,-0.018181,-0.036381,-0.135259,0.017777,...,-0.030397,0.57278,0.05254,-0.308745,-0.091502,0.39261,0.340

In [0]:
file = '/content/drive/My Drive/Colab Notebooks/Thinkful/Module 34 - Final Capstone/data/CDC Detection/funnel_1_model.sav'

pickle.dump(lr, open(file, 'wb'))

In [0]:
save_df = pd.concat([X_tst,y_tst], axis=1)

write_path = '/content/drive/My Drive/Colab Notebooks/Thinkful/Module 34 - Final Capstone/data/CDC Detection/m1_test_set.csv'

save_df.to_csv(write_path)